In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import time
import random

import os
import sys
sys.path.append('../src')

from math_utils import *
from model import *
from utils import *

## Data

In [20]:
DATA_PATH = '../Concrete_Data.xls' # link dataset: https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength
SEED = 42
df_data = pd.read_excel(DATA_PATH).sample(frac=1, random_state=SEED)

In [21]:
df_data

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
31,266.00,114.0,0.00,228.00,0.00,932.0,670.00,365,52.908320
109,362.60,189.0,0.00,164.90,11.60,944.7,755.80,7,55.895819
136,389.90,189.0,0.00,145.90,22.00,944.7,755.80,28,74.497882
88,362.60,189.0,0.00,164.90,11.60,944.7,755.80,3,35.301171
918,145.00,0.0,179.00,202.00,8.00,824.0,869.00,28,10.535193
...,...,...,...,...,...,...,...,...,...
87,286.30,200.9,0.00,144.70,11.20,1004.6,803.70,3,24.400556
330,246.83,0.0,125.08,143.30,11.99,1086.8,800.89,14,42.216615
466,190.34,0.0,125.18,166.61,9.88,1079.0,798.90,100,33.563692
121,475.00,118.8,0.00,181.10,8.90,852.1,781.50,28,68.299493


In [22]:
data = df_data.to_numpy() #converting from excel to numpy array(meanwhile shuffling)

dev_len, test_len = 0.8*(len(data)), 0.2*(len(data)) # 80% for dev and 20% for test
dev_set, test_set = data[:int(dev_len)], data[int(dev_len):]

train_len, val_len = 0.8*(len(dev_set)), 0.2*(len(dev_set)) # 80% for train and 20% for validation
train_set, val_set = dev_set[:int(train_len)], dev_set[int(train_len):]

In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_set[:, :-1] = scaler.fit_transform(train_set[:, :-1])
val_set[:, :-1] = scaler.transform(val_set[:, :-1])
test_set[:, :-1] = scaler.transform(test_set[:, :-1])

In [24]:
len(train_set), len(val_set), len(test_set) # 80% for train, 10% for validation and 10% for test

(659, 165, 206)

## Model Selection

In [43]:
def grid_search(config_dict:dict, train_set:np.ndarray, val_set:np.ndarray, config_trials:int=5, metric:callable=mse) -> tuple:
    """
    Perform grid search over the given configuration dictionary.

    Parameters
    ----------
    config_dict : dict
        Dictionary containing the configurations to be tested.
    config_trials : int
        Number of trials for each configuration.

    Returns
    -------
    config_dict : dataframe
        all configurations and their mean and variance of MSE
    """

    metric_name = metric.__name__ if hasattr(metric, '__name__') else str(metric)

    df_dict = {'hidden_size': [], 'activation': [], 'init_method': [], 'init_params': [],
               f'train_{metric_name}': [], f'train_{metric_name}_var': [], f'val_{metric_name}': [], f'val_{metric_name}_var': []}
    # create all possible combination of the configurations
    keys, values = zip(*config_dict.items())
    configs = [dict(zip(keys, v)) for v in itertools.product(*values)]
    for config in configs:
        df_dict['hidden_size'].append(config['hidden_size'])
        df_dict['activation'].append(config['activation'])
        df_dict['init_method'].append(config['init_method'])
        df_dict['init_params'].append(config['init_params'])

        val_values = []
        train_values = []
        for _ in range(config_trials):
            model = ELM(input_size=config['input_size'], hidden_size=config['hidden_size'], hidden_activation=config['activation'], 
                        init_method=config['init_method'], init_params=config['init_params'])
            model.fit(train_set[:, :-1], train_set[:, -1].reshape(-1, 1))

            pred = model.predict(train_set[:, :-1])
            train_values.append(metric(train_set[:, -1], pred))

            pred = model.predict(val_set[:, :-1])
            val_values.append(metric(val_set[:, -1], pred))

        df_dict[f'train_{metric_name}'].append(np.mean(train_values))
        df_dict[f'train_{metric_name}_var'].append(np.var(train_values))
        df_dict[f'val_{metric_name}'].append(np.mean(val_values))
        df_dict[f'val_{metric_name}_var'].append(np.var(val_values))

    
    config_df = pd.DataFrame(df_dict)
    return config_df


In [44]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

config_dict = {
    'input_size': [8],
    'hidden_size': [100, 150, 200],
    'activation': [np.tanh, sigmoid],
    'init_method': ['uniform'],
    'init_params': [(-0.5, 0.5), (-0.4, 0.4)]
}

results = grid_search(config_dict, train_set, val_set, config_trials=5, metric=mae)

In [45]:
results = results.sort_values(by=f'val_{mae.__name__}', ascending=True)
results

,hidden_size,activation,init_method,init_params,train_mae,train_mae_var,val_mae,val_mae_var
3,100,<function sigmoid at 0x7fccb968e710>,uniform,"(-0.4, 0.4)",5.222868,0.011332,5.940390,0.037829
7,150,<function sigmoid at 0x7fccb968e710>,uniform,"(-0.4, 0.4)",4.556559,0.001963,6.054613,0.020812
2,100,<function sigmoid at 0x7fccb968e710>,uniform,"(-0.5, 0.5)",5.200434,0.004815,6.065953,0.039716
6,150,<function sigmoid at 0x7fccb968e710>,uniform,"(-0.5, 0.5)",4.592908,0.009389,6.086930,0.031810
11,200,<function sigmoid at 0x7fccb968e710>,uniform,"(-0.4, 0.4)",4.000403,0.004378,6.109327,0.047316
10,200,<function sigmoid at 0x7fccb968e710>,uniform,"(-0.5, 0.5)",3.967236,0.007985,6.251513,0.073713
9,200,<ufunc 'tanh'>,uniform,"(-0.4, 0.4)",11.716636,0.217418,19.155088,0.111215
4,150,<ufunc 'tanh'>,uniform,"(-0.5, 0.5)",13.644420,0.306348,19.966415,0.855585
8,200,<ufunc 'tanh'>,uniform,"(-0.5, 0.5)",11.790978,0.128740,20.197455,0.632735
5,150,<ufunc 'tanh'>,uniform,"(-0.4, 0.4)",14.022739,0.168362,20.245816,1.523241


In [52]:
config_dict = {
    'input_size': [8],
    'hidden_size': [100, 200, 300, 400, 500],
    'activation': [np.tanh, sigmoid],
    'init_method': ['normal'],
    'init_params': [(0, 0.1), (0, 0.5), (0, 1)]
}

results = grid_search(config_dict, train_set, val_set, config_trials=5, metric=mae)

In [54]:
results = results.sort_values(by=f'val_{mae.__name__}', ascending=True)
results

,hidden_size,activation,init_method,init_params,train_mae,train_mae_var,val_mae,val_mae_var
4,100,<function sigmoid at 0x7fccb968e710>,normal,"(0, 0.5)",5.135749,0.006814,5.853018,0.032565
3,100,<function sigmoid at 0x7fccb968e710>,normal,"(0, 0.1)",5.162837,0.011785,6.052965,0.046362
5,100,<function sigmoid at 0x7fccb968e710>,normal,"(0, 1)",5.296747,0.028817,6.096542,0.203885
10,200,<function sigmoid at 0x7fccb968e710>,normal,"(0, 0.5)",3.869024,0.009656,6.214591,0.059117
11,200,<function sigmoid at 0x7fccb968e710>,normal,"(0, 1)",4.138291,0.028916,6.284729,0.027656
9,200,<function sigmoid at 0x7fccb968e710>,normal,"(0, 0.1)",3.926173,0.007903,6.403538,0.123948
17,300,<function sigmoid at 0x7fccb968e710>,normal,"(0, 1)",2.894673,0.005860,6.838635,0.041082
16,300,<function sigmoid at 0x7fccb968e710>,normal,"(0, 0.5)",2.832542,0.003486,7.089176,0.451987
15,300,<function sigmoid at 0x7fccb968e710>,normal,"(0, 0.1)",2.776614,0.018924,9.172546,1.232793
23,400,<function sigmoid at 0x7fccb968e710>,normal,"(0, 1)",1.898903,0.008780,9.587326,0.327478
